### Sri Lanka Road Optimization
This notebook is part of a Sri Lanka tourism project that measures the distance from airports to district capitals to toursim destinations. 

#### Workflow
- part 1: OD Airport to major cities
- part 2: OD Major cities to one another
- part 3: OD Major cities to tourism destinations
- part 4: OD using mapbox traffic api OR mapbox roads data w/ traffic (2-3x for different times of day)
- part 5*: Identify optimal edges for road rehabilitation (traffic travel time is divergent)
- part 6*: OD - simulate road improvements with adjusted cost
- part 7*: identify possible edges for improvement (highest value for money). 
#### *To-do


#### to run this script you must already have the G_time.pickle (from data folder)
#### note: this script and the architecture of gostnets uses a lot of memory. for example, creating nodes_gdf took ~70% of a 64GB memory.


In [1]:
# import modules and dependencies

import geopandas as gpd
import pandas as pd
import os, sys, time
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), r"C:\Users\wb557957\Desktop\GOSTnets-master"))

import GOSTnets as gn
from scipy import *
import GOSTnets.network_clean as gnClean
import importlib
import networkx as nx
import osmnx as ox
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

In [2]:
pth = r"C:\Users\wb557957\Desktop\LKA"
inPth = r"C:\Users\wb557957\Desktop\LKA\lka_data"
outPth = r"C:\Users\wb557957\Desktop\LKA\lka_outputs"

In [4]:
G_time = nx.read_gpickle(os.path.join(pth, r'G_largestGraph_osmid_time.pickle'))

In [6]:
edge_gdf = pd.read_csv('sri_lanka_hwy_w_traffic1.csv',  delimiter=",")
node_gdf = pd.read_csv('G_osmid_nodes.csv',  delimiter=",")

'node_gdf = pd.read_csv(\'G_osmid_nodes.csv\',  delimiter=",")'

In [7]:
# read in snapped origins and destinations
airportdf = pd.read_csv('airport_snapped_osmid.csv',  delimiter=",")
citydf = pd.read_csv('city_snapped_osmid.csv',  delimiter=",")
tourismdf = pd.read_csv('tourism_snapped_osmid.csv',  delimiter=",")

In [8]:
# NN - osm id of nearest network node.
airportdf.head()

,Unnamed: 0,osm_id,emergencyh,aeroway,operatorty,source,capacitype,addrfull,emergency,name,addrcity,building,geometry,x,y,NN,NN_dist
0,0,2.700505e+09,NaN,gate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (79.8846214999823 7.176073400367159),79.884621,7.176073,5916588300,191.443706


In [10]:
citydf.head()

,Unnamed: 0,osm_id,code,fclass,population,name,country,geometry,x,y,NN,NN_dist
0,0,44240758.0,1001,city,27500,Nuwara Eliya,LKA,POINT (80.76706730021579 6.973815700434272),80.767067,6.973816,4784044133,1.631987
1,1,49374844.0,1001,city,17900,Kegalle,LKA,POINT (80.3454131999593 7.253200599927766),80.345413,7.253201,3794050817,9.364111
2,2,50794342.0,1005,national_capital,753000,Colombo,LKA,POINT (79.85384630026658 6.934996899587304),79.853846,6.934997,5385270047,6.505032
3,3,247220841.0,1001,city,120000,Hambantota,LKA,POINT (81.12425630008823 6.124912599836648),81.124256,6.124913,1814540688,47.809655
4,4,258049423.0,1001,city,40800,Matale,LKA,POINT (80.62341730016834 7.472125099893503),80.623417,7.472125,1727494683,82.604133


In [11]:
tourismdf.head()

,Unnamed: 0,Loc_name,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,...,USER_langC,USER_adm1c,USER_adm2c,USER_adm3c,USER_adm4c,geometry,NN,NN_dist,x,y
0,0,World,M,80.01,A,Embekka Katharagama Dewalaya,"Embekka Katharagama Dewalaya, Embekka, LKA",Embekka Katharagama Dewalaya,POI,Temple,...,ENG,LK2,LK21,NaN,NaN,POINT (80.56806000000006 7.218110000000024),1743126554,14.430893,80.568060,7.218110
1,1,NaN,M,100.00,PP,"x=80.742936, y=7.400645",NaN,NaN,NaN,NaN,...,ENG,LK2,LK21,NaN,NaN,POINT (80.74293550533594 7.400644946810061),3645433570,80.094863,80.742936,7.400645
2,2,World,M,87.33,A,Royal Palace Park-Kandy,"Royal Palace Park-Kandy, Keerthie Sri Rajasing...",Royal Palace Park-Kandy,POI,Park,...,ENG,LK2,LK21,NaN,NaN,POINT (80.63751000000013 7.289470000000051),3804266901,7.492898,80.637510,7.289470
3,3,World,M,75.48,M,Kandy,"Kandy, LKA",Kandy,Locality,City,...,ENG,LK2,LK21,NaN,NaN,POINT (80.63866000000007 7.294970000000049),5507321512,29.207204,80.638660,7.294970
4,4,World,M,65.07,A,Bauddha Bala Mandalaya,"Bauddha Bala Mandalaya, Akkara Road, Ranawana ...",Bauddha Bala Mandalaya,POI,Temple,...,ENG,LK2,LK21,NaN,NaN,POINT (80.62270000000007 7.328350000000057),1809637836,69.904258,80.622700,7.328350


In [13]:
# create lists of OD's
airport_ls = list(set(list(airportdf.NN)))
city_ls = list(set(list(citydf.NN)))
tourism_ls = list(set(list(tourismdf.NN)))

### part 1: OD airport to major cities

In [14]:
# output is a numpy array of OD time values
# 'AC' stands for Airport-to-City
OD_AC = gn.calculate_OD(G_time,
                     airport_ls, 
                     city_ls, 
                     fail_value = 9999999)

OD_AC

array([[ 39.64850111, 129.39232447, 142.6434966 ,  81.1454507 ,
         60.4884683 , 128.33726623, 120.46487756,  74.00586124,
         59.09294624,  18.30620227, 161.54588862, 104.96826004,
         59.963836  , 100.20283328,   7.42715748, 184.13301429,
         14.41596862, 110.8415158 , 124.8304607 , 174.17964904,
         44.87393341,  90.91354579, 169.75843849,  41.22356656,
        134.05577029, 159.78532664]])

In [15]:
# get shape of OD matrix to confirm count of O&D
OD_AC.shape

# 26 observations

(1, 26)

In [16]:
# convert the OD Matrix numpy array into a pandas dataframe using minutes as the measure
# output: minutes from airport to each destination NN id (n=26)
OD_AC = OD_AC / 60

OD_dfAC = pd.DataFrame(OD_AC, columns = city_ls, index = airport_ls)

OD_dfAC

,3794050817,4137095171,4071257605,3379679501,5300884110,1814540688,1945604886,1727494683,4140985758,5385270047,...,3289453904,2822277587,7098666963,6130006236,4944055519,4784044133,1732894187,1138325745,1243386867,1681288951
5916588300,0.660808,2.156539,2.377392,1.352424,1.008141,2.138954,2.007748,1.233431,0.984882,0.305103,...,0.240266,1.847359,2.080508,2.902994,0.747899,1.515226,2.829307,0.687059,2.234263,2.663089


In [17]:
# transpose the df and save as an object 
OD_dfAC = OD_dfAC.transpose()

In [18]:
# print transposed OD
OD_dfAC

,5916588300
3794050817,0.660808
4137095171,2.156539
4071257605,2.377392
3379679501,1.352424
5300884110,1.008141
1814540688,2.138954
1945604886,2.007748
1727494683,1.233431
4140985758,0.984882
5385270047,0.305103


In [ ]:
OD_dfAC.to_csv(r'OD_df_airport2city.csv')

### part 2: OD cities to other cities

In [19]:
# in part 1, we've already imported, snapped and created a list from our district capitals (cities) dataset.
# in this process, we will call the cities into the OD calculation as both origin and destiation
# 'CC' stands for City-to-City
OD_CC = gn.calculate_OD(G_time,
                     city_ls, 
                     city_ls, 
                     fail_value = 9999999)

OD_CC

array([[  0.        , 104.93059842, 108.88976928, 103.98497507,
         51.11305189, 118.96184982,  81.63142228,  34.35736014,
         64.50565385,  41.47025122, 121.89738751, 127.80778441,
         20.31533489,  77.01003675,  40.3010907 , 183.03489006,
         28.7291934 ,  83.66685615, 130.2307742 , 149.71792298,
         17.49431892,  51.26504468, 145.29671244,  64.06309093,
        106.88111064, 120.13682553],
       [104.93059842,   0.        , 165.42223615, 199.90633832,
        154.4029636 , 217.68472892, 147.29627118,  82.41782207,
         71.03439397, 137.39161448, 149.95621147, 223.72914767,
         96.51675082,  29.53984775, 123.4578246 ,  78.10731995,
        124.65055666,  77.64773462,  42.29619891,  44.79035287,
         87.43669269, 135.58351959,  40.36914233, 159.98445419,
         53.22185228, 122.57156774],
       [108.88976928, 165.42223615,   0.        , 125.55391466,
         82.15937957,  60.15105088,  31.68983129,  95.43276375,
        155.56029625, 130.0934

In [20]:
OD_CC.shape

(26, 26)

In [21]:
OD_CC = OD_CC / 60

OD_dfCC = pd.DataFrame(OD_CC, columns = city_ls, index = city_ls)

OD_dfCC

,3794050817,4137095171,4071257605,3379679501,5300884110,1814540688,1945604886,1727494683,4140985758,5385270047,...,3289453904,2822277587,7098666963,6130006236,4944055519,4784044133,1732894187,1138325745,1243386867,1681288951
3794050817,0.000000,1.748843,1.814829,1.733083,0.851884,1.982697,1.360524,0.572623,1.075094,0.691171,...,0.478820,1.394448,2.170513,2.495299,0.291572,0.854417,2.421612,1.067718,1.781352,2.002280
4137095171,1.748843,0.000000,2.757037,3.331772,2.573383,3.628079,2.454938,1.373630,1.183907,2.289860,...,2.077509,1.294129,0.704937,0.746506,1.457278,2.259725,0.672819,2.666408,0.887031,2.042859
4071257605,1.814829,2.757037,0.000000,2.092565,1.369323,1.002518,0.528164,1.590546,2.592672,2.168224,...,2.170123,1.552040,3.299367,3.503493,1.809173,1.036938,3.429806,2.059075,2.392491,1.413470
3379679501,1.733083,3.331772,2.092565,0.000000,1.355189,1.155141,2.224764,2.305706,2.234205,1.060330,...,1.313840,2.977377,3.329830,4.078228,1.874501,2.323018,4.004541,0.665365,3.364281,3.461296
5300884110,0.851884,2.573383,1.369323,1.355189,0.000000,1.130886,1.134918,1.424507,1.899634,0.798974,...,0.800872,2.218987,2.995052,3.319838,1.116111,1.254698,3.246151,0.689824,2.605891,2.574158
1814540688,1.982697,3.628079,1.002518,1.155141,1.130886,0.000000,1.175642,2.386481,3.030447,1.929787,...,1.931686,2.502799,4.125866,4.374534,2.246925,1.503054,4.300847,1.820364,3.350269,2.371248
1945604886,1.360524,2.454938,0.528164,2.224764,1.134918,1.175642,0.000000,1.251749,2.191757,1.906366,...,1.820578,1.329658,2.997267,3.201393,1.408259,0.508774,3.127706,1.824670,2.308300,1.441740
1727494683,0.572623,1.373630,1.590546,2.305706,1.424507,2.386481,1.251749,0.000000,1.283699,1.263794,...,1.051443,1.019235,1.915960,2.120086,0.503669,0.886095,2.046399,1.640341,1.406139,1.777997
4140985758,1.075094,1.183907,2.592672,2.234205,1.899634,3.030447,2.191757,1.283699,0.000000,1.186857,...,1.126813,1.603451,1.095706,1.930362,0.783522,1.774939,1.856675,1.568840,1.664691,2.466815
5385270047,0.691171,2.289860,2.168224,1.060330,0.798974,1.929787,1.906366,1.263794,1.186857,0.000000,...,0.271928,1.935465,2.282482,3.036316,0.832589,1.431347,2.962629,0.394965,2.322369,2.693451


In [22]:
OD_dfCC = OD_dfCC.transpose()

In [23]:
OD_dfCC 

,3794050817,4137095171,4071257605,3379679501,5300884110,1814540688,1945604886,1727494683,4140985758,5385270047,...,3289453904,2822277587,7098666963,6130006236,4944055519,4784044133,1732894187,1138325745,1243386867,1681288951
3794050817,0.000000,1.748843,1.814829,1.733083,0.851884,1.982697,1.360524,0.572623,1.075094,0.691171,...,0.478820,1.394448,2.170513,2.495299,0.291572,0.854417,2.421612,1.067718,1.781352,2.002280
4137095171,1.748843,0.000000,2.757037,3.331772,2.573383,3.628079,2.454938,1.373630,1.183907,2.289860,...,2.077509,1.294129,0.704937,0.746506,1.457278,2.259725,0.672819,2.666408,0.887031,2.042859
4071257605,1.814829,2.757037,0.000000,2.092565,1.369323,1.002518,0.528164,1.590546,2.592672,2.168224,...,2.170123,1.552040,3.299367,3.503493,1.809173,1.036938,3.429806,2.059075,2.392491,1.413470
3379679501,1.733083,3.331772,2.092565,0.000000,1.355189,1.155141,2.224764,2.305706,2.234205,1.060330,...,1.313840,2.977377,3.329830,4.078228,1.874501,2.323018,4.004541,0.665365,3.364281,3.461296
5300884110,0.851884,2.573383,1.369323,1.355189,0.000000,1.130886,1.134918,1.424507,1.899634,0.798974,...,0.800872,2.218987,2.995052,3.319838,1.116111,1.254698,3.246151,0.689824,2.605891,2.574158
1814540688,1.982697,3.628079,1.002518,1.155141,1.130886,0.000000,1.175642,2.386481,3.030447,1.929787,...,1.931686,2.502799,4.125866,4.374534,2.246925,1.503054,4.300847,1.820364,3.350269,2.371248
1945604886,1.360524,2.454938,0.528164,2.224764,1.134918,1.175642,0.000000,1.251749,2.191757,1.906366,...,1.820578,1.329658,2.997267,3.201393,1.408259,0.508774,3.127706,1.824670,2.308300,1.441740
1727494683,0.572623,1.373630,1.590546,2.305706,1.424507,2.386481,1.251749,0.000000,1.283699,1.263794,...,1.051443,1.019235,1.915960,2.120086,0.503669,0.886095,2.046399,1.640341,1.406139,1.777997
4140985758,1.075094,1.183907,2.592672,2.234205,1.899634,3.030447,2.191757,1.283699,0.000000,1.186857,...,1.126813,1.603451,1.095706,1.930362,0.783522,1.774939,1.856675,1.568840,1.664691,2.466815
5385270047,0.691171,2.289860,2.168224,1.060330,0.798974,1.929787,1.906366,1.263794,1.186857,0.000000,...,0.271928,1.935465,2.282482,3.036316,0.832589,1.431347,2.962629,0.394965,2.322369,2.693451


In [27]:
OD_dfCC.to_csv(r'OD_df_city2city.csv')

In [22]:
index=city_ls

pd.DataFrame.sort_values(OD_dfCC, by=index, ascending=True)

,59905,237826,164101,262918,19847,281879,210210,48165,215717,106667,...,257368,68568,285275,315998,117093,45673,64874,136052,151421,328574
59905,0.000000,355.135884,145.582929,522.096118,416.906767,478.113338,430.048766,547.230303,243.344348,328.640533,...,69.461088,288.529008,230.294625,421.795377,361.979672,398.525645,299.907625,116.432549,371.010554,325.888692
257368,69.461088,306.283970,76.121842,489.366483,347.445679,445.383704,360.587678,477.769216,173.883260,304.214521,...,0.000000,219.067920,206.794746,352.334289,329.817774,329.064557,230.446538,58.642656,301.549466,256.427604
136052,116.432549,314.298686,84.136558,497.381200,355.460396,453.398420,368.602395,485.783932,146.299286,312.229238,...,58.642656,227.082637,214.809463,344.359209,337.832491,337.079274,238.461255,0.000000,277.488314,264.442321
136272,119.145355,245.913149,77.656585,412.873383,331.049291,368.890603,330.974782,456.121118,175.418003,219.417798,...,117.440068,202.671532,121.071890,335.937901,252.756937,312.668169,214.050150,143.567974,297.953506,240.031216
164101,145.582929,230.167656,0.000000,413.250170,271.329365,369.267390,284.471364,401.652902,97.761598,228.098207,...,76.121842,142.951607,130.678432,276.217976,253.701460,252.948244,154.330224,84.136558,225.427804,180.311290
230721,199.976941,176.413342,54.399540,359.495855,231.533813,315.513076,233.027318,358.173655,107.262365,174.343893,...,130.515854,103.156054,76.924118,236.422423,199.947146,213.152691,114.534672,138.530570,198.438028,140.515738
285275,230.294625,124.850143,130.678432,291.810377,242.145874,247.827597,209.934686,335.081022,183.541257,98.354792,...,206.794746,176.928026,0.000000,310.194394,131.693931,196.771504,141.789416,214.809463,272.209999,128.636727
215717,243.344348,233.746200,97.761598,416.828714,254.998333,372.845934,287.944311,370.345670,0.000000,240.393956,...,173.883260,108.082322,183.541257,203.382677,257.280005,256.421190,157.803171,146.299286,136.511782,183.784237
68568,288.529008,191.025828,142.951607,349.639336,146.938962,330.125562,245.223938,277.262499,108.082322,197.673584,...,219.067920,0.000000,176.928026,133.289320,214.559632,185.146547,115.082798,227.082637,95.304925,141.063864
64874,299.907625,116.187741,154.330224,299.270254,138.350506,255.287475,157.467591,265.175338,157.803171,131.488945,...,230.446538,115.082798,141.789416,199.702427,139.721545,98.618020,0.000000,238.461255,196.439825,25.981066


### part 3: OD cities to destinations
in part 3, we calculate travel time for all cities to all destinations

there were are a total of 414 geocoded sites, but only 336 had unique nearest nodes.


In [28]:
# calculate OD from cities to tourism destinations
# these routes will be called later to compare with travel time with traffic.
# 'CT' stands for City-to-Tourism
OD_CT = gn.calculate_OD(G_time,
                     city_ls, 
                     tourism_ls, 
                     fail_value = 9999999)

OD_CT

array([[ 63.87047664,  42.84099437,  51.19892705, ...,  76.86608272,
        120.78754182,  42.39570342],
       [159.79183989, 138.76235763, 135.51740196, ...,  31.07508205,
        123.41396682, 138.31706668],
       [123.35187679, 128.6873615 ,  62.25547012, ..., 145.25886998,
         85.45893034, 129.40814014],
       ...,
       [  1.44262735,  22.28574619,  99.49839261, ..., 130.95739851,
        184.85063275,  22.73657873],
       [161.74235211, 140.71286985, 137.46791418, ...,  59.92213374,
         75.00912729, 140.26757889],
       [184.00730217, 162.97781991, 108.03703919, ..., 109.28619214,
          2.28694837, 162.53252895]])

In [29]:
OD_CT.shape

(26, 342)

In [30]:
# convert the OD Matrix numpy array into a pandas dataframe using minutes as the measure
# output: minutes from airport (18713) to each destination NN id (n=336)
OD_CT = OD_CT / 60

OD_dfCT = pd.DataFrame(OD_CT, columns = tourism_ls, index = city_ls)

OD_dfCT

,1667080206,6991570964,6117715989,1743126554,5130592302,2100457519,1185558575,130445367,2085208123,4047450172,...,3772792790,1730404311,258523099,1685802983,1691551721,3216934896,1506164725,1657419767,1675585534,4621961215
3794050817,1.064508,0.714017,0.853315,0.320008,2.362439,2.596546,0.830663,1.451822,1.205143,0.347842,...,0.773612,2.132089,1.286036,2.011502,2.059995,0.430506,1.742047,1.281101,2.013126,0.706595
4137095171,2.663197,2.312706,2.258623,1.758351,3.703245,0.926615,2.429352,3.050512,2.611864,1.598690,...,2.372302,3.318332,1.185718,1.225831,2.527642,2.172632,3.340737,0.517918,2.056899,2.305284
4071257605,2.055865,2.144789,1.037591,1.615108,0.990948,3.440544,2.078324,2.358783,1.006434,1.483203,...,2.097543,0.606034,1.660189,2.279848,0.882131,1.765616,2.098819,2.420981,1.424316,2.156802
3379679501,0.689409,1.036794,2.323671,2.053091,1.641529,4.179475,0.937042,0.281261,1.869442,2.080925,...,0.980971,1.491018,2.868965,3.594431,2.974696,1.560585,0.009115,2.847988,3.472141,1.044308
5300884110,0.686614,0.775539,1.255351,1.171892,1.510627,3.421085,0.709074,1.073928,0.668644,1.199726,...,0.728293,1.280277,2.110576,2.836042,2.251454,0.627866,1.364153,2.105641,2.585003,0.787552
1814540688,1.817427,1.906352,1.503708,2.081224,0.546995,4.398322,1.839887,1.436260,1.292572,2.161701,...,1.859106,0.396484,2.610948,3.237626,1.884648,1.758679,1.161395,3.236454,2.382094,1.918365
1945604886,1.821460,1.910385,0.509427,1.086944,1.271860,3.302641,1.843920,2.124378,0.772030,1.045190,...,1.863139,0.939557,1.437807,2.274640,1.272154,1.237452,2.231018,2.118882,1.452585,1.922398
1727494683,1.637131,1.286639,0.884993,0.384720,2.482700,2.221333,1.403285,2.024445,1.238234,0.225060,...,1.346235,2.151841,0.910823,1.636289,1.835711,0.912945,2.314670,1.037574,1.788842,1.279218
4140985758,1.565630,1.211917,1.773837,1.265680,3.371545,2.036017,1.331785,1.952944,2.127078,1.154669,...,1.274869,3.131314,1.495040,2.003491,2.836964,1.498883,2.243169,0.681256,2.480855,1.202281
5385270047,0.391755,0.039668,1.432000,1.011179,2.309528,3.137563,0.157909,0.779069,1.447372,1.039013,...,0.103457,2.079178,1.827053,2.552519,2.751166,0.668914,1.069294,1.806076,2.704297,0.017781


In [ ]:
OD_dfCT= OD_dfCT.transpose()

In [ ]:
OD_dfCT

In [ ]:
OD_dfCT.to_csv(r'OD_df_city2tourism.csv')

In [34]:
# delete things *after saving* to clear up memory.
#del edge_gdf
#del airportdf
#del citydf
#del tourismdf
del OD_dfAC
del OD_dfCC
del OD_dfCT

In [ ]:
%reset

### part 4: collect traversed paths
#### example from gn iceland tutorial - https://github.com/worldbank/GOSTnets/blob/master/Tutorials/EXAMPLE%20Finding_links_between_pairs.ipynb

#### 4.1: get routes from airports to cities

#### 4.2: get routes from cities to cities

#### 4.3: get routes from cities to tourism destinations

In [29]:
# re-read in G_time
G_time = nx.read_gpickle(os.path.join(pth, 'lka_outputs', r'G_time.pickle'))
# make sure origins and destinations have been snapped to network and created as list objects.

In [30]:
# create separate object with G_time nodes and edges:
nodes_gdf = gn.node_gdf_from_graph(G_time)
edges_gdf = gn.edge_gdf_from_graph(G_time)

In [31]:
# example of one OD pair
# create object of all nodes in shortest path
obj_nodes = nx.shortest_path(G_time, 
                             source=city_ls[0], 
                             target=tourism_ls[0], 
                             weight='time')
print(city_ls[0])
print(tourism_ls[0])
# this is a list of the network graph nodes that, connected, make up the shortest path from the first origin and first destination
print(obj_nodes)


59905
42497
[59905, 178723, 257367, 119234, 91554, 149298, 176520, 229307, 49500, 184671, 164349, 258571, 136969, 178905, 180099, 212250, 210359, 257737, 210191, 176519, 313423, 194153, 240362, 100152, 111419, 296965, 231717, 221662, 110592, 176187, 21871, 205395, 259259, 78204, 309080, 326997, 88357, 213846, 264207, 79091, 2102, 231208, 92176, 154091, 294690, 140709, 227299, 28664, 189276, 161030, 184116, 110907, 286116, 323478, 213579, 230494, 39361, 137697, 207094, 290389, 166143, 255778, 156605, 321771, 261197, 72694, 226265, 38017, 197436, 129117, 124574, 303760, 230931, 70100, 310539, 155131, 324667, 152607, 81373, 113929, 165099, 169677, 270659, 63712, 96967, 59998, 157702, 80538, 171738, 210444, 125557, 61345, 4420, 119668, 243206, 245100, 189192, 259933, 226346, 326718, 171996, 75519, 52506, 271159, 151212, 64717, 26263, 245260, 201232, 250851, 91826, 159538, 45952, 311600, 50583, 256254, 301869, 63548, 198663, 260424, 6361, 166325, 223984, 147881, 44414, 310185, 72366, 124195

In [32]:
??nx.shortest_path

when running the following code block, the script stops at the first OD pair

In [33]:
# calculate line strings connecting all origins to all destinations
import time

all_res=[]
all_connections=[]
oIdx = 0 #city ID count

print('start: %s\n' % time.ctime())
for c in city_ls: # n=26
    oIdx = oIdx + 1
    print(f'{oIdx} of {len(city_ls)}')
    for t in tourism_ls: # n=336
        obj_nodes = nx.shortest_path(G_time, source=c, target=t, weight="time")
        all_edges = []
        start_node = obj_nodes[0]
        end_node = obj_nodes[-1]
        for idx in range(0, len(obj_nodes) - 1):
            start_node = obj_nodes[idx]
            end_node = obj_nodes[idx + 1]
            cur_edge = edges_gdf.loc[(edges_gdf['stnode'] == start_node) & (edges_gdf['endnode'] == end_node), 'geometry'].iloc[0]
            all_edges.append(cur_edge)
        #    all_connections.append([start_node, end_node, cur_edge])
       # all_res.append([c, t, MultiLineString(all_edges)])
        print('\nend: %s' % time.ctime())

start: Fri Mar 13 14:37:08 2020

1 of 26

end: Fri Mar 13 14:37:40 2020

end: Fri Mar 13 14:37:47 2020

end: Fri Mar 13 14:37:49 2020

end: Fri Mar 13 14:38:11 2020

end: Fri Mar 13 14:38:13 2020

end: Fri Mar 13 14:38:14 2020

end: Fri Mar 13 14:38:14 2020

end: Fri Mar 13 14:38:21 2020

end: Fri Mar 13 14:38:43 2020

end: Fri Mar 13 14:39:14 2020

end: Fri Mar 13 14:39:55 2020

end: Fri Mar 13 14:40:37 2020

end: Fri Mar 13 14:41:20 2020

end: Fri Mar 13 14:41:20 2020

end: Fri Mar 13 14:41:23 2020

end: Fri Mar 13 14:41:24 2020

end: Fri Mar 13 14:42:07 2020

end: Fri Mar 13 14:42:10 2020

end: Fri Mar 13 14:42:57 2020

end: Fri Mar 13 14:43:41 2020

end: Fri Mar 13 14:44:02 2020

end: Fri Mar 13 14:44:30 2020

end: Fri Mar 13 14:45:11 2020

end: Fri Mar 13 14:45:50 2020

end: Fri Mar 13 14:46:31 2020

end: Fri Mar 13 14:47:12 2020

end: Fri Mar 13 14:47:36 2020

end: Fri Mar 13 14:48:06 2020

end: Fri Mar 13 14:48:14 2020

end: Fri Mar 13 14:48:31 2020

end: Fri Mar 13 14:48:40 202


end: Fri Mar 13 16:07:27 2020

end: Fri Mar 13 16:07:46 2020

end: Fri Mar 13 16:07:53 2020

end: Fri Mar 13 16:08:16 2020

end: Fri Mar 13 16:08:32 2020

end: Fri Mar 13 16:08:58 2020

end: Fri Mar 13 16:09:44 2020

end: Fri Mar 13 16:10:10 2020

end: Fri Mar 13 16:11:01 2020

end: Fri Mar 13 16:11:31 2020

end: Fri Mar 13 16:12:14 2020

end: Fri Mar 13 16:12:57 2020

end: Fri Mar 13 16:13:42 2020

end: Fri Mar 13 16:14:05 2020

end: Fri Mar 13 16:14:05 2020

end: Fri Mar 13 16:14:34 2020

end: Fri Mar 13 16:14:51 2020

end: Fri Mar 13 16:14:52 2020

end: Fri Mar 13 16:15:05 2020

end: Fri Mar 13 16:15:06 2020

end: Fri Mar 13 16:15:14 2020

end: Fri Mar 13 16:16:04 2020

end: Fri Mar 13 16:16:19 2020

end: Fri Mar 13 16:16:20 2020

end: Fri Mar 13 16:16:40 2020

end: Fri Mar 13 16:17:21 2020

end: Fri Mar 13 16:17:23 2020

end: Fri Mar 13 16:17:24 2020

end: Fri Mar 13 16:18:11 2020

end: Fri Mar 13 16:18:11 2020

end: Fri Mar 13 16:18:13 2020

end: Fri Mar 13 16:18:14 2020

end: Fr


end: Fri Mar 13 17:05:21 2020

end: Fri Mar 13 17:05:29 2020

end: Fri Mar 13 17:05:36 2020

end: Fri Mar 13 17:05:39 2020

end: Fri Mar 13 17:05:43 2020

end: Fri Mar 13 17:05:52 2020

end: Fri Mar 13 17:06:06 2020

end: Fri Mar 13 17:06:29 2020

end: Fri Mar 13 17:06:30 2020

end: Fri Mar 13 17:06:42 2020

end: Fri Mar 13 17:06:59 2020

end: Fri Mar 13 17:07:17 2020

end: Fri Mar 13 17:07:44 2020

end: Fri Mar 13 17:08:10 2020

end: Fri Mar 13 17:08:29 2020

end: Fri Mar 13 17:08:54 2020

end: Fri Mar 13 17:09:21 2020

end: Fri Mar 13 17:09:45 2020

end: Fri Mar 13 17:09:46 2020

end: Fri Mar 13 17:09:47 2020

end: Fri Mar 13 17:10:14 2020

end: Fri Mar 13 17:10:20 2020

end: Fri Mar 13 17:10:29 2020

end: Fri Mar 13 17:10:35 2020

end: Fri Mar 13 17:10:43 2020

end: Fri Mar 13 17:11:12 2020

end: Fri Mar 13 17:11:20 2020

end: Fri Mar 13 17:11:23 2020

end: Fri Mar 13 17:11:37 2020

end: Fri Mar 13 17:11:49 2020

end: Fri Mar 13 17:12:13 2020

end: Fri Mar 13 17:12:30 2020

end: Fr


end: Fri Mar 13 18:04:38 2020

end: Fri Mar 13 18:05:10 2020

end: Fri Mar 13 18:05:11 2020

end: Fri Mar 13 18:05:12 2020

end: Fri Mar 13 18:05:45 2020

end: Fri Mar 13 18:05:45 2020

end: Fri Mar 13 18:06:03 2020

end: Fri Mar 13 18:06:22 2020

end: Fri Mar 13 18:06:22 2020

end: Fri Mar 13 18:06:52 2020

end: Fri Mar 13 18:07:10 2020

end: Fri Mar 13 18:07:37 2020

end: Fri Mar 13 18:08:13 2020

end: Fri Mar 13 18:08:25 2020

end: Fri Mar 13 18:08:53 2020

end: Fri Mar 13 18:08:55 2020

end: Fri Mar 13 18:08:56 2020

end: Fri Mar 13 18:08:57 2020

end: Fri Mar 13 18:08:58 2020

end: Fri Mar 13 18:08:58 2020

end: Fri Mar 13 18:09:05 2020

end: Fri Mar 13 18:09:06 2020

end: Fri Mar 13 18:09:06 2020

end: Fri Mar 13 18:09:12 2020

end: Fri Mar 13 18:09:18 2020

end: Fri Mar 13 18:09:18 2020

end: Fri Mar 13 18:09:53 2020

end: Fri Mar 13 18:10:26 2020

end: Fri Mar 13 18:10:43 2020

end: Fri Mar 13 18:11:16 2020

end: Fri Mar 13 18:11:40 2020

end: Fri Mar 13 18:11:58 2020

end: Fr


end: Fri Mar 13 19:11:50 2020

end: Fri Mar 13 19:12:05 2020

end: Fri Mar 13 19:12:06 2020

end: Fri Mar 13 19:13:11 2020

end: Fri Mar 13 19:13:24 2020

end: Fri Mar 13 19:13:25 2020

end: Fri Mar 13 19:13:27 2020

end: Fri Mar 13 19:13:30 2020

end: Fri Mar 13 19:13:33 2020

end: Fri Mar 13 19:13:56 2020

end: Fri Mar 13 19:14:19 2020

end: Fri Mar 13 19:14:56 2020

end: Fri Mar 13 19:14:56 2020

end: Fri Mar 13 19:14:56 2020

end: Fri Mar 13 19:15:01 2020

end: Fri Mar 13 19:15:04 2020

end: Fri Mar 13 19:15:09 2020

end: Fri Mar 13 19:15:10 2020

end: Fri Mar 13 19:15:11 2020

end: Fri Mar 13 19:15:50 2020

end: Fri Mar 13 19:16:19 2020

end: Fri Mar 13 19:16:19 2020

end: Fri Mar 13 19:17:05 2020

end: Fri Mar 13 19:17:46 2020

end: Fri Mar 13 19:17:47 2020

end: Fri Mar 13 19:18:19 2020

end: Fri Mar 13 19:19:05 2020

end: Fri Mar 13 19:19:20 2020

end: Fri Mar 13 19:19:22 2020

end: Fri Mar 13 19:20:00 2020

end: Fri Mar 13 19:20:02 2020

end: Fri Mar 13 19:21:10 2020

end: Fr

...


end: Sat Mar 14 16:13:45 2020

end: Sat Mar 14 16:13:58 2020

end: Sat Mar 14 16:14:04 2020

end: Sat Mar 14 16:14:05 2020

end: Sat Mar 14 16:14:05 2020

end: Sat Mar 14 16:14:28 2020

end: Sat Mar 14 16:14:28 2020

end: Sat Mar 14 16:14:28 2020

end: Sat Mar 14 16:14:29 2020

end: Sat Mar 14 16:14:39 2020

end: Sat Mar 14 16:14:42 2020

end: Sat Mar 14 16:15:01 2020

end: Sat Mar 14 16:15:22 2020

end: Sat Mar 14 16:15:24 2020

end: Sat Mar 14 16:15:36 2020

end: Sat Mar 14 16:15:39 2020

end: Sat Mar 14 16:15:39 2020

end: Sat Mar 14 16:15:53 2020

end: Sat Mar 14 16:16:18 2020

end: Sat Mar 14 16:16:20 2020

end: Sat Mar 14 16:16:24 2020

end: Sat Mar 14 16:16:34 2020

end: Sat Mar 14 16:16:34 2020

end: Sat Mar 14 16:16:34 2020

end: Sat Mar 14 16:16:35 2020

end: Sat Mar 14 16:16:59 2020

end: Sat Mar 14 16:17:07 2020

end: Sat Mar 14 16:17:15 2020

end: Sat Mar 14 16:17:22 2020

end: Sat Mar 14 16:17:46 2020

end: Sat Mar 14 16:18:01 2020

end: Sat Mar 14 16:18:17 2020

end: Sa


end: Sat Mar 14 17:27:32 2020

end: Sat Mar 14 17:27:32 2020

end: Sat Mar 14 17:28:03 2020

end: Sat Mar 14 17:28:31 2020

end: Sat Mar 14 17:28:54 2020

end: Sat Mar 14 17:28:55 2020

end: Sat Mar 14 17:29:01 2020

end: Sat Mar 14 17:29:46 2020

end: Sat Mar 14 17:30:01 2020

end: Sat Mar 14 17:30:03 2020

end: Sat Mar 14 17:30:03 2020

end: Sat Mar 14 17:30:26 2020

end: Sat Mar 14 17:31:02 2020

end: Sat Mar 14 17:31:04 2020

end: Sat Mar 14 17:31:49 2020

end: Sat Mar 14 17:32:33 2020

end: Sat Mar 14 17:33:15 2020

end: Sat Mar 14 17:33:35 2020

end: Sat Mar 14 17:33:46 2020

end: Sat Mar 14 17:33:53 2020

end: Sat Mar 14 17:33:53 2020

end: Sat Mar 14 17:33:54 2020

end: Sat Mar 14 17:33:56 2020

end: Sat Mar 14 17:34:18 2020

end: Sat Mar 14 17:34:56 2020

end: Sat Mar 14 17:35:03 2020

end: Sat Mar 14 17:35:04 2020

end: Sat Mar 14 17:35:44 2020

end: Sat Mar 14 17:35:49 2020

end: Sat Mar 14 17:35:55 2020

end: Sat Mar 14 17:35:56 2020

end: Sat Mar 14 17:36:16 2020

end: Sa


end: Sat Mar 14 18:34:44 2020

end: Sat Mar 14 18:34:47 2020

end: Sat Mar 14 18:34:49 2020

end: Sat Mar 14 18:35:21 2020

end: Sat Mar 14 18:35:24 2020

end: Sat Mar 14 18:35:28 2020

end: Sat Mar 14 18:35:28 2020

end: Sat Mar 14 18:35:31 2020

end: Sat Mar 14 18:35:43 2020

end: Sat Mar 14 18:35:47 2020

end: Sat Mar 14 18:35:47 2020

end: Sat Mar 14 18:36:18 2020

end: Sat Mar 14 18:36:57 2020

end: Sat Mar 14 18:37:34 2020

end: Sat Mar 14 18:37:37 2020

end: Sat Mar 14 18:38:13 2020

end: Sat Mar 14 18:38:16 2020

end: Sat Mar 14 18:38:19 2020

end: Sat Mar 14 18:38:19 2020

end: Sat Mar 14 18:38:22 2020

end: Sat Mar 14 18:38:24 2020

end: Sat Mar 14 18:38:24 2020

end: Sat Mar 14 18:38:25 2020

end: Sat Mar 14 18:38:52 2020

end: Sat Mar 14 18:39:33 2020

end: Sat Mar 14 18:39:36 2020

end: Sat Mar 14 18:39:38 2020

end: Sat Mar 14 18:39:39 2020

end: Sat Mar 14 18:39:42 2020

end: Sat Mar 14 18:39:43 2020

end: Sat Mar 14 18:39:47 2020

end: Sat Mar 14 18:40:23 2020

end: Sa


end: Sat Mar 14 19:25:44 2020

end: Sat Mar 14 19:25:53 2020

end: Sat Mar 14 19:25:59 2020

end: Sat Mar 14 19:26:02 2020

end: Sat Mar 14 19:26:11 2020

end: Sat Mar 14 19:26:27 2020

end: Sat Mar 14 19:26:38 2020

end: Sat Mar 14 19:26:38 2020

end: Sat Mar 14 19:26:56 2020

end: Sat Mar 14 19:27:05 2020

end: Sat Mar 14 19:27:12 2020

end: Sat Mar 14 19:27:21 2020

end: Sat Mar 14 19:27:33 2020

end: Sat Mar 14 19:27:42 2020

end: Sat Mar 14 19:27:42 2020

end: Sat Mar 14 19:27:47 2020

end: Sat Mar 14 19:27:56 2020

end: Sat Mar 14 19:28:01 2020

end: Sat Mar 14 19:28:23 2020

end: Sat Mar 14 19:28:27 2020

end: Sat Mar 14 19:28:34 2020

end: Sat Mar 14 19:28:42 2020

end: Sat Mar 14 19:28:42 2020

end: Sat Mar 14 19:28:47 2020

end: Sat Mar 14 19:28:55 2020

end: Sat Mar 14 19:28:56 2020

end: Sat Mar 14 19:29:12 2020

end: Sat Mar 14 19:29:21 2020

end: Sat Mar 14 19:29:21 2020

end: Sat Mar 14 19:29:29 2020

end: Sat Mar 14 19:29:47 2020

end: Sat Mar 14 19:29:47 2020

end: Sa


end: Sat Mar 14 20:07:24 2020

end: Sat Mar 14 20:07:39 2020

end: Sat Mar 14 20:07:39 2020

end: Sat Mar 14 20:07:47 2020

end: Sat Mar 14 20:07:52 2020

end: Sat Mar 14 20:08:11 2020

end: Sat Mar 14 20:08:22 2020

end: Sat Mar 14 20:08:26 2020


In [ ]:
# write all routes to csv file
all_results = pd.DataFrame(all_res, columns['O', 'D', 'geometry'])
all_results.to_csv(os.path.join(pth, 'lka_data', 'OD_links_city2tourism.csv'))

In [ ]:
# tabulate use of individual links and write to csv file
all_conn = pd.DataFrame(all_connections, columns=['start','node','geometry'])
all_connections_count = pd.DataFrame(all_conn.groupby(['start','node']).count())
all_connections_count.reset_index(inplace=True)
all_connections_first = pd.DataFrame(all_conn.groupby(['start','node']).first())
all_connections_first.reset_index(inplace=True)
all_connections_first['count'] = all_connections_count['geometry']

In [ ]:
all_connections_first.to_csv(os.path.join(pth, "tutorial_data","OD_links_usage.csv"))
